<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/deep_learning/Pho2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

PATH ='drive/My Drive/data'
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sat May  2 11:42:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

In [0]:
import joblib
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np  
import tensorflow.compat.v1 as tf  
import re 
import sys
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split
from IPython.display import clear_output 
from ipywidgets import interact 



    
def ipa_(x):

  x=x.replace('ɑ̃','0')
  x=x.replace('ʁ','r')  
  x=x.replace('ɔ̃','1') 
  x=x.replace('ɛ̃', '2') 
  x=x.replace('œ̃','3') 
  x=x.replace('tr','4')
  x=x.replace('kr','5')
  x=x.replace('dr','6')
  x=x.replace('gr','7') 
  x=x.replace('pr','8') 
  x=x.replace('br','9')
  
  return x 
def ipa_reverse(x) : 


  x=x.replace('0','ɑ̃')
  x=x.replace('1','ɔ̃') 
  x=x.replace( '2','ɛ̃') 
  x=x.replace('3','œ̃') 
  x=x.replace('4','tr')
  x=x.replace('5','kr')
  x=x.replace('6','dr')
  x=x.replace('7','gr') 
  x=x.replace('8','pr') 
  x=x.replace('9','br')
  x=x.replace('pad>','>')
  x=x.replace('pad<','<')
  
  return x 

# Import data 


In [3]:


dfdic=joblib.load(PATH+'/phono/final_data.job') 
print('dfdic is original data ')

for key in dfdic.keys():
    dfdic[key]['valeur phono enfant']=dfdic[key]['valeur phono enfant'].apply(ipa_).apply(lambda x:list(x) )

dfdic is original data 


In [0]:
data_test = dfdic['antoine']['valeur phono enfant'][dfdic['antoine'].age==3]

# General way of making Pho2Vec with skip gram methodology  
Inspired by this article : 
https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac

In [0]:
class Pho2Vec():
    
    def __init__(self,data,name):
        self.vocab=set(np.sum(data))
        self.vocab_len= len(self.vocab) 
        self.name= name 
        self.model =None
        self.data=data
        self.pho_vec={}
        self.pho2int={} 
        self.int2pho={}
        self.history={}
        self.vector_representation = None 
        for i,word in enumerate(self.vocab):
            self.pho2int[word] = i
            self.int2pho[i] = word 
        self.pho2int['pad<']=self.vocab_len
        self.pho2int['pad>']=self.vocab_len+1 
        self.int2pho[self.vocab_len]='pad<' 
        self.int2pho[self.vocab_len+1]='pad>'  
        self.vocab_len += 2
        self.vocab_len_input=self.vocab_len 
        self.vocab_len_output=self.vocab_len

    

    def dataforw2c(self,windows_size): 
        self.windows_size=windows_size
        self.target=[]  
        self.context=[]
        start='pad<' 
        end ='pad>' 
        try : 
            self.data = data.values 
        except : 
            pass 
        for talk in self.data: 
            for idx,pho in enumerate(talk):  
                tmp_context=[]
                self.target.append(pho) 
                for ct in range(idx-self.windows_size,idx+self.windows_size+1,1):  
                    if ct == idx : 
                        continue  
                    elif ct < 0 : 
                        tmp_context.append(start) 
                    elif ct > len(talk)-1  : 
                        tmp_context.append(end)
                    else  :  
                        try : 
                            tmp_context.append(talk[ct]) 
                        except : 
                            AttributeError('An error occured during padding or sequencing , check that all your data are array or list') 
                self.context.append(tmp_context)  
        # Transform the list to array to facilitate manipulation
        self.context=np.array(self.context)
        self.target=np.array(self.target)
    


        
        # function to convert numbers to one hot vectors
    def __to_one_hot(self,X):
        X = np.vectorize(lambda X: self.pho2int[X])(X) # We create an numpy function to transform to int each value in the array 
        return  np.eye(self.vocab_len)[X]  # We return the associate one hot array 


    def train_test(self,train_size=0.9): 
        # Use of the sklearn train_test split method . Improvement can be done by only numpy method here  
        self.x= self.__to_one_hot(self.target)  # Hot encoded vector
        self.y= self.__to_one_hot(self.context)  # Hot encoded vector 
        # We split the data for having a validation set .
        self.x_train,self.x_test,self.y_train,self.y_test =train_test_split(self.x,
                                                                           self.y, 
                                                                           train_size=train_size) 
        
        
    
    
    
    
    
    
    
    
    
    def model_2vec(self,embedding_dim=6,n_iter=10000,batch_size=16): 
        try : 
            x_train = self.x_train 
            x_test  = self.x_test 

            y_train = self.y_train 
            y_test = self.y_test  
        except : 
            AttributeError('train and test set must be initialize')
        
        
        x = tf.placeholder(tf.float32, shape=(None, self.vocab_len_input))   
        
        
        self.embedding_dim=embedding_dim
        W1 = tf.Variable(tf.random_normal([self.vocab_len_input, self.embedding_dim]))
        b1 = tf.Variable(tf.random_normal([self.embedding_dim]))
        W2={} 
        b2={} 
        prediction=[]
        cross_entropy_loss=[]
        y_label=[]

        hidden_layer = tf.add(tf.matmul(x,W1), b1)
        for i in range(self.windows_size*2):
            y_label.append(tf.placeholder(tf.float32, shape=(None, self.vocab_len_output)))
            W2[i] = tf.Variable(tf.random_normal([self.embedding_dim, self.vocab_len_output]))
            b2[i] = tf.Variable(tf.random_normal([self.vocab_len_output]))
            prediction.append(tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2[i]), b2[i])))
            cross_entropy_loss.append(tf.reduce_mean(-tf.reduce_sum(y_label[i] * tf.log(tf.clip_by_value(prediction[i],1e-10,1.0)), reduction_indices=[1])))
        
        total_loss=tf.reduce_sum(cross_entropy_loss)


       

       
        sess = tf.Session()
     
       
        Adam_opti=tf.train.AdamOptimizer(learning_rate=0.001,  beta1=0.9, beta2=0.999,   epsilon=1e-08, use_locking=False,name='Adam') 
        #train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)  
        train_step=Adam_opti.minimize(total_loss) 

        init = tf.global_variables_initializer()
        sess.run(init) 
        
       


       
        # train for n_iter iterations
        self.history['loss']=[]
        self.history['validation_loss']=[] 


        #Let's define batch size first and how pass batch to the graph : 
        self.batch_size= batch_size 
        N=self.x_train.shape[0] 
        #Let's compute how many batch we can pass in one epoch  
        nbatch= N // self.batch_size   
        # Number of element isn't necessary a multiple of the batch size , so the last feed to the graph will be the last element 
        # There is two way of solving this problem but i'll do the easy one , for the last iteration i pass the remaining element by list comprehension 

        for i in range(n_iter):
            
            for pos in range(nbatch): 
                start = pos*self.batch_size 
                end = (pos+1)*self.batch_size
                _,loss = sess.run([train_step,total_loss], feed_dict={x: x_train[start:end], **{y_label[i]: y_train[start:end,i,:]  for i in range(self.windows_size*2)}}    )

            val_loss= total_loss.eval(session=sess, feed_dict={x: x_test, **{y_label[i]: y_test[:,i,:]  for i in range(self.windows_size*2)}}  )  
            self.history['loss'].append(loss) 
            self.history['validation_loss'].append(val_loss)
        
            if i % 10 == 0 : 
                self.vector_representation= W1.eval(session=sess) + b1.eval(session=sess)
                try : 
                    sys.stdout.write(f'progression : {i} / {n_iter}  val loss={val_loss}  loss = {loss} with {self.name}') 
                    clear_output(wait=True)
                except : 
                    sys.stdout.write(str_loss)
                
        
            #tbar.refresh()
        def get_pho_vectors(self): 
            try : 
                self.vectors={}
                for key,val in self.pho2int.items():
                    self.vectors[key]=self.vector_representation[val] 
            except : 
                AttributeError('train model_2vec first')
            



In [0]:
#Création des objets pour chaque enfant, par age  
Colaj_vec = {}
for key,val in dfdic.items() :  
    Colaj_vec[key]= {age : Pho2Vec(val['valeur phono enfant'][val.age==age ],f'{key}_{int(age)}') for age in np.unique(val.age) if (age >1 and age <5 ) }  
    # We only want to keep age beetween 1 and 5 for now, but for further studies we can evaluate on greater or lower age .




In [24]:
# Initialisation des objets pour chaque enfant par age   
windows_size = 1   # ( +- 1 phoneme autour du phoneme cible )  
train_size = 0.95    # ( On conserve 95 % des données , le but ici est de garder 5% pour voir si l'apprentissage est cohérent) 
for key,val in Colaj_vec.items(): 
    for key2,val2 in val.items(): 
        val2.dataforw2c(windows_size=windows_size) 
        val2.train_test(train_size=train_size) 
        print(f'{key} at age {int(key2)}  --> prepared for train data shape : {val2.x_train.shape}')

antoine at age 2  --> prepared for train data shape : (10048, 42)
antoine at age 3  --> prepared for train data shape : (4826, 43)
antoine at age 4  --> prepared for train data shape : (3660, 38)
theophile at age 2  --> prepared for train data shape : (5507, 41)
theophile at age 3  --> prepared for train data shape : (9332, 43)
theophile at age 4  --> prepared for train data shape : (4293, 40)
julie at age 2  --> prepared for train data shape : (9578, 41)
julie at age 3  --> prepared for train data shape : (5479, 41)
julie at age 4  --> prepared for train data shape : (2486, 39)
anae at age 2  --> prepared for train data shape : (9551, 42)
anae at age 3  --> prepared for train data shape : (5591, 42)
anae at age 4  --> prepared for train data shape : (4657, 41)
madeleine at age 2  --> prepared for train data shape : (11948, 43)
madeleine at age 3  --> prepared for train data shape : (5654, 40)
madeleine at age 4  --> prepared for train data shape : (4920, 40)
adrien at age 2  --> prepa

In [26]:
# Training, for treaning each dataset we decide to make 5000 iterations with  size = 512 for batch size  
for key,val in Colaj_vec.items(): 
    for key,val in val.items(): 
        print(f'Training for {key} at age {key2}')  
        # For embedding dimension a rule of thumb is  vocab_size**0.25 , here there is almost 40 element in vocabulary 
        # So  Embedding_dim = 40*0.25 = 2.51 => we rount it at 3  

        val.model_2vec(embedding_dim=3 , n_iter = 10000 ,batch_size=512 )

progression : 9990 / 10000  val loss=4.004584789276123  loss = 3.8979098796844482 with adrien_4

In [38]:
Colaj_vec['adrien'][4].vector_representation.shape 
len(Colaj_vec['adrien'][4].int2pho)

38

In [0]:
VECTOR={}  
for key,val in Colaj_vec.items() :  
    age={}
    for key2,val2 in val.items() :  
        vect={} 
        V=val2.vector_representation
        for key3,val3 in val2.pho2int.items(): 
            vect[ipa_reverse(key3)] =V[val3]
        age[key2]=vect 
    VECTOR[key]=age


In [70]:
joblib.dump(VECTOR,PATH +'/Vectorized_colaj.job')

['drive/My Drive/data/Vectorized_colaj.job']

In [0]:
from scipy.cluster.hierarchy import dendrogram, linkage

PATH2= 'drive/My Drive/data/dendrogramme/'

In [0]:
for child , val in VECTOR.items(): 
    for age,val2 in val.items(): 
        sample_test=pd.DataFrame(val2).T
                


        l = linkage(sample_test, method='complete', metric='euclidean',optimal_ordering=True)

        # calculate full dendrogram
        plt.figure(figsize=(15, 8))
        plt.title(f'Dendrogramme pour proximité des phonèmes par vectorization et Clustering Hierarchique \n distance: euclidienne , methode : complete , \n enfant: {child.title()}  à {age} ans' )
        plt.ylabel('distance')
        plt.xlabel('phoneme')

        dendrogram(
            l,
            leaf_rotation=0.,  # rotates the x axis labels
            leaf_font_size=10.,  # font size for the x axis labels
            orientation='top',
            labels=sample_test.index)
        plt.yscale('linear')
        plt.savefig( PATH2+f'dendogramme_{child}_{age}.png')
        plt.close()

In [0]:

import os 
from PIL import Image
dendro = os.listdir(PATH2)

In [120]:
plt.rcParams['figure.figsize']=(20,10)
@interact 
def show_img(x=dendro):  
    fig, ax = plt.subplots()
    
    ax.imshow(Image.open(PATH2+x))
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

interactive(children=(Dropdown(description='x', options=('dendogramme_antoine_2.0.png', 'dendogramme_antoine_3…